In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from numpy.linalg import inv
from sklearn import preprocessing

In [236]:
df = pd.read_csv('train_data.csv')
df

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,point
0,1565607600,30.329370,59.940488,63,4,52,6,0,0,0,0,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,1565568000,30.342845,59.869542,44,0,40,0,4,0,0,0,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,1565568000,30.322632,60.039332,30,0,77,7,0,0,0,1,0101000020E61000009E01B90998523E40888919D70805...
3,1565568000,30.322632,59.938238,13,0,634,17,32,0,0,7,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,1565568000,30.309158,59.932612,8,0,0,0,0,0,0,0,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...,...,...,...,...,...,...,...,...,...,...
8526521,1565978400,30.324878,59.932612,48,0,15,2,0,0,0,1,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,1565978400,30.324878,59.932612,20,1,0,0,0,0,0,0,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,1565978400,30.324878,59.932612,54,2,75,7,0,0,0,6,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,1565971200,30.324878,59.932612,15,0,350,4,25,0,0,0,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [238]:
# сократим количество потребляемой оперативы
df = df[['timestamp', 'point']]
df

,timestamp,point
0,1565607600,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,1565568000,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,1565568000,0101000020E61000009E01B90998523E40888919D70805...
3,1565568000,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,1565568000,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...
8526521,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,1565971200,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [239]:
len(df.point.unique())

7041

In [240]:
# validation contains 710 cubes
# February 2020
valid_df = pd.read_csv('valid.csv')
valid_df = valid_df[['point', 'hour']]
valid_df

,point,hour
0,0101000020E6100000202C93AD71513E4070B5CC78A7F6...,1582448400
1,0101000020E6100000C5FE0F8FE53D3E4033438EB75DF6...,1582225200
2,0101000020E6100000AB00D6E0B14B3E40CB88A75E4B00...,1581260400
3,0101000020E61000005DECCB372B533E40E4A1F770CEF7...,1581958800
4,0101000020E6100000669634C7914E3E407AB3D8B484F7...,1582779600
...,...,...
704,0101000020E61000006AEBE80E454C3E407F614299EAF3...,1580659200
705,0101000020E61000006141807FDE503E402B348A98ADF9...,1581048000
706,0101000020E6100000268147F5244F3E407AB3D8B484F7...,1581656400
707,0101000020E61000008EADE7EACA5B3E40CAE971F73AF7...,1580724000


In [241]:
len(valid_df.point.unique())

155

In [242]:
true_point = sorted(valid_df.point.unique())
true_point

['0101000020E610000000000000000000000000000000000000',
 '0101000020E61000000C830D47F15C3E400A41C338F1F64D40',
 '0101000020E61000000C830D47F15C3E40CAE971F73AF74D40',
 '0101000020E61000000FD8C18EA45A3E408991431816F74D40',
 '0101000020E61000000FD8C18EA45A3E40CAE971F73AF74D40',
 '0101000020E6100000132D76D657583E408991431816F74D40',
 '0101000020E61000001868CE7F1F343E40975BBA5723FE4D40',
 '0101000020E6100000199C86BCF6773E401FBF4913EFF54D40',
 '0101000020E6100000199C86BCF6773E406A1274D638F64D40',
 '0101000020E61000001CD7DE65BE533E40A554BF083DF84D40',
 '0101000020E61000002012370F862F3E40A08C1570AEEA4D40',
 '0101000020E6100000202C93AD71513E4009249E9F54F24D40',
 '0101000020E6100000202C93AD71513E401E66D608FBFB4D40',
 '0101000020E6100000202C93AD71513E4041010E3DD6EB4D40',
 '0101000020E6100000202C93AD71513E4054F844A221FD4D40',
 '0101000020E6100000202C93AD71513E4070B5CC78A7F64D40',
 '0101000020E6100000202C93AD71513E407A441D895BF54D40',
 '0101000020E6100000202C93AD71513E40CEB8CE2B18F84D40',
 '01010000

In [246]:
# Оставим в датасете только интересующие нас кубы и переведем timestamp в даты
df['timestamp']= [datetime.fromtimestamp(x) for x in df['timestamp']]
df_clear = df[df.point.isin(true_point)]
df_clear

TypeError: 'Timestamp' object cannot be interpreted as an integer

In [259]:
df_clear

,timestamp,point
0,2019-08-12 14:00:00,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,2019-08-12 03:00:00,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40888919D70805...
3,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,2019-08-12 03:00:00,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...
8526521,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,2019-08-16 19:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [249]:
#df_clear = df_clear.groupby(['point_label', 'timestamp']).size().reset_index(name='counts')
working_df = df_clear.groupby(['point', 'timestamp']).size().reset_index(name='counts')
# working_df = df_clear.groupby(['point','timestamp']).count().unstack(fill_value=
#                                                          0
#                                                          ).stack()
working_df

,point,timestamp,counts
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2
3,0101000020E61000000000000000000000000000000000...,2019-01-01 08:00:00,3
4,0101000020E61000000000000000000000000000000000...,2019-01-01 09:00:00,2
...,...,...,...
896276,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 17:00:00,2
896277,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 18:00:00,1
896278,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 19:00:00,2
896279,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 22:00:00,1


In [266]:
working_df = df_clear.groupby(['point','timestamp']).size().unstack(fill_value=
                                                         0
                                                         ).stack()
working_df = working_df.reset_index(name='counts')
working_df

,point,timestamp,counts
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2
3,0101000020E61000000000000000000000000000000000...,2019-01-01 06:00:00,0
4,0101000020E61000000000000000000000000000000000...,2019-01-01 07:00:00,0
...,...,...,...
1473115,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 22:00:00,1
1473116,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 23:00:00,0
1473117,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 00:00:00,2
1473118,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 01:00:00,0


In [267]:
# Разделим timestamp на дату и время
working_df['Dates'] = pd.to_datetime(working_df['timestamp']).dt.date
working_df['Time'] = pd.to_datetime(working_df['timestamp']).dt.time
working_df

,point,timestamp,counts,Dates,Time
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3,2019-01-01,03:00:00
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2,2019-01-01,04:00:00
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2,2019-01-01,05:00:00
3,0101000020E61000000000000000000000000000000000...,2019-01-01 06:00:00,0,2019-01-01,06:00:00
4,0101000020E61000000000000000000000000000000000...,2019-01-01 07:00:00,0,2019-01-01,07:00:00
...,...,...,...,...,...
1473115,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 22:00:00,1,2020-01-31,22:00:00
1473116,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 23:00:00,0,2020-01-31,23:00:00
1473117,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 00:00:00,2,2020-02-01,00:00:00
1473118,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 01:00:00,0,2020-02-01,01:00:00


In [269]:
working_df.timestamp.unique()

array(['2019-01-01T03:00:00.000000000', '2019-01-01T04:00:00.000000000',
       '2019-01-01T05:00:00.000000000', ...,
       '2020-02-01T00:00:00.000000000', '2020-02-01T01:00:00.000000000',
       '2020-02-01T02:00:00.000000000'], dtype='datetime64[ns]')

In [268]:
points = working_df.point.unique()
points

array(['0101000020E610000000000000000000000000000000000000',
       '0101000020E61000000C830D47F15C3E400A41C338F1F64D40',
       '0101000020E61000000C830D47F15C3E40CAE971F73AF74D40',
       '0101000020E61000000FD8C18EA45A3E408991431816F74D40',
       '0101000020E61000000FD8C18EA45A3E40CAE971F73AF74D40',
       '0101000020E6100000132D76D657583E408991431816F74D40',
       '0101000020E61000001868CE7F1F343E40975BBA5723FE4D40',
       '0101000020E6100000199C86BCF6773E401FBF4913EFF54D40',
       '0101000020E6100000199C86BCF6773E406A1274D638F64D40',
       '0101000020E61000001CD7DE65BE533E40A554BF083DF84D40',
       '0101000020E61000002012370F862F3E40A08C1570AEEA4D40',
       '0101000020E6100000202C93AD71513E4009249E9F54F24D40',
       '0101000020E6100000202C93AD71513E401E66D608FBFB4D40',
       '0101000020E6100000202C93AD71513E4041010E3DD6EB4D40',
       '0101000020E6100000202C93AD71513E4054F844A221FD4D40',
       '0101000020E6100000202C93AD71513E4070B5CC78A7F64D40',
       '0101000020E61000

In [270]:
len(points)

155

In [271]:
example = working_df[working_df.point == points[0]]
example

,point,timestamp,counts,Dates,Time
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3,2019-01-01,03:00:00
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2,2019-01-01,04:00:00
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2,2019-01-01,05:00:00
3,0101000020E61000000000000000000000000000000000...,2019-01-01 06:00:00,0,2019-01-01,06:00:00
4,0101000020E61000000000000000000000000000000000...,2019-01-01 07:00:00,0,2019-01-01,07:00:00
...,...,...,...,...,...
9499,0101000020E61000000000000000000000000000000000...,2020-01-31 22:00:00,8,2020-01-31,22:00:00
9500,0101000020E61000000000000000000000000000000000...,2020-01-31 23:00:00,13,2020-01-31,23:00:00
9501,0101000020E61000000000000000000000000000000000...,2020-02-01 00:00:00,13,2020-02-01,00:00:00
9502,0101000020E61000000000000000000000000000000000...,2020-02-01 01:00:00,0,2020-02-01,01:00:00


In [178]:
valid_df['time'] = [datetime.fromtimestamp(x) for x in valid_df['hour']]
valid_df

,point,hour,point_label,time
0,0101000020E6100000202C93AD71513E4070B5CC78A7F6...,1582448400,15,2020-02-23 12:00:00
1,0101000020E6100000C5FE0F8FE53D3E4033438EB75DF6...,1582225200,120,2020-02-20 22:00:00
2,0101000020E6100000AB00D6E0B14B3E40CB88A75E4B00...,1581260400,108,2020-02-09 18:00:00
3,0101000020E61000005DECCB372B533E40E4A1F770CEF7...,1581958800,52,2020-02-17 20:00:00
4,0101000020E6100000669634C7914E3E407AB3D8B484F7...,1582779600,60,2020-02-27 08:00:00
...,...,...,...,...
704,0101000020E61000006AEBE80E454C3E407F614299EAF3...,1580659200,62,2020-02-02 19:00:00
705,0101000020E61000006141807FDE503E402B348A98ADF9...,1581048000,54,2020-02-07 07:00:00
706,0101000020E6100000268147F5244F3E407AB3D8B484F7...,1581656400,20,2020-02-14 08:00:00
707,0101000020E61000008EADE7EACA5B3E40CAE971F73AF7...,1580724000,76,2020-02-03 13:00:00
